In [2]:
import pandas as pd
import sweetviz as sv

# Load the dataset (example: iris dataset)
df_1 = pd.read_csv('datasets/CBP2019.CB1900CBP-2023-05-14T012245.csv')
df_2 = pd.read_csv('datasets/Bachelor_Degree_Majors.csv')
df_3 = pd.read_csv('datasets/state_regions.csv')


# Drop any columns from df_1 that we do not need for our analysis
columns_to_drop = ["Year (YEAR)", "Meaning of NAICS code (NAICS2017_LABEL)", "2017 NAICS code (NAICS2017)",
                   "Meaning of Legal form of organization code (LFO_LABEL)"]
df_1 = df_1.drop(columns_to_drop, axis=1)


# Since we dropped some columns, some rows have become duplicates of others. Thus, we proceed to drop them.
df_1 = df_1.drop_duplicates()


# Rename the columns of df_1 to make them easier to work with
column_rename_mapping = {
    "Geographic Area Name (NAME)": "State",
    "Meaning of Employment size of establishments code (EMPSZES_LABEL)": "Business size",
    "Number of establishments (ESTAB)": "#Establishments",
    "Annual payroll ($1,000) (PAYANN)": "Average annual payroll",
    "First-quarter payroll ($1,000) (PAYQTR1)": "Average first-quarter payroll",
    "Number of employees (EMP)": "Average #employees"
}
df_1.rename(columns=column_rename_mapping, inplace=True)


# Contradiction mitigation: For the CPB dataset (df_1), drop any rows where "Business size" == "All establishments"
df_1 = df_1[(df_1["Business size"] != "All establishments")]

# Contradiction mitigatino: For the Bachelor's dataset, replace all "25 and older" values of the "Age Group" column
# with the value "younger than 25"
df_2['Age Group'] = df_2['Age Group'].replace('25 and older', 'younger than 25')


# For the CPB dataset (df_1), only keep the rows where the value of the "Business size" attribute refers to a company
# that represents a "major" competitor, according to our client's criteria
values_to_keep = [
    "Establishments with 50 to 99 employees",
    "Establishments with 100 to 249 employees",
    "Establishments with 250 to 499 employees",
    "Establishments with 500 to 999 employees",
    "Establishments with 1,000 employees or more"
]
df_1 = df_1[df_1["Business size"].isin(values_to_keep)]


# Get the non-common values between the "State" columns of the 2 datasets (df_1 and df_2)
symmetric_difference = pd.Series(list(set(df_1['State']).symmetric_difference(set(df_2['State']))))
print("\n> States included in CBP but not in Bachelor Majors dataset: {}".format(len(symmetric_difference)))
print(symmetric_difference)


# We will to drop the rows in df_1 and df_2 that contain any of the non-common (State) values.
df_1 = df_1[~df_1['State'].isin(symmetric_difference.tolist())]
df_2 = df_2[~df_2['State'].isin(symmetric_difference.tolist())]


# Remove "," from all numeric values in the CPB dataframe
# Loop through each column in the DataFrame
for column in df_1.columns:
    # Remove commas from values
    df_1[column] = df_1[column].str.replace(',', '')

# Remove "," from all numeric values in the Bachelor's dataframe
# Loop through each column in the DataFrame
for column in df_2.columns:
    # Remove commas from values
    df_2[column] = df_2[column].str.replace(',', '')

# Convert df_1 number columns to numeric values
numeric_columns = ["#Establishments", "Average annual payroll", "Average first-quarter payroll",
                   "Average #employees"]
df_1[numeric_columns] = df_1[numeric_columns].apply(pd.to_numeric)

# Convert df_2 number columns to numeric values
numeric_columns = ["Bachelor's Degree Holders", "Science and Engineering", "Science and Engineering Related Fields",
                   "Business", "Education", "Arts, Humanities and Others"]
df_2[numeric_columns] = df_2[numeric_columns].apply(pd.to_numeric)


# Add the information from the "State Regions" dataset (df_3) to CBP (df_1) as a new column
df_1 = pd.merge(df_1, df_3, on='State', how='left')

# ====== Generate a new "Men to women degree holders ratio" column in df_1 that contains the men to women
# bachelor holders ratio for each state
# Filter the DataFrame for "Male" and "Female" separately
male_df = df_2[df_2['Sex'] == 'Male']
female_df = df_2[df_2['Sex'] == 'Female']

# Group by "State" and calculate the sum of "Bachelor's Degree Holders" for each gender
male_counts = male_df.groupby('State')['Bachelor\'s Degree Holders'].sum()
female_counts = female_df.groupby('State')['Bachelor\'s Degree Holders'].sum()

# Calculate the ratio of men to women degree holdersfor each state
ratio = male_counts / female_counts

# Create a new column in df_1 and map the men/women ratios there based on the "State" value of each entry.
df_1['Men to women degree holders ratio'] = df_1['State'].map(ratio)


# == Determine the field that has the largest and second-largest number of graduates per State
# Filter the dataset to keep only rows where "Sex" is equal to "Total"
filtered_df = df_2[df_2["Sex"] == "Total"]

# Group the filtered DataFrame by "State"
grouped_df = filtered_df.groupby("State")

# Create an empty dictionary to store the results
state_column_dict = {}

# == Determine the most popular field of studies
# Iterate over each distinct value of "State"
for state, group in grouped_df:
    # Calculate the summed values for each column
    summed_values = group[["Science and Engineering", "Science and Engineering Related Fields", "Business", "Education",
                           "Arts, Humanities and Others"]].sum()

    # Find the column with the highest summed value
    max_column = summed_values.idxmax()

    # Save the column name to the state:column dictionary
    state_column_dict[state] = max_column

# Add the new column to the df_2 dataframe
df_1["Most popular degree field"] = df_1["State"].map(state_column_dict)

# ==== Determine the 2nd most popular field of studies
# Iterate over each distinct value of "State"
for state, group in grouped_df:
    # Calculate the summed values for each column
    summed_values = group[["Science and Engineering", "Science and Engineering Related Fields", "Business", "Education",
                           "Arts, Humanities and Others"]].sum()

    # Sort the summed values in descending order and get the column name with the second largest summed value
    second_largest_column = summed_values.sort_values(ascending=False).index[1]

    # Save the column name to the state:column dictionary
    state_column_dict[state] = second_largest_column

# Add the new column to the df_2 dataframe
df_1["2nd Most popular degree field"] = df_1["State"].map(state_column_dict)


# ====== Add a new "#(Mid)Senior degree holders" column to df_1 that is generated by summing the values of
# "Bachelor's Degree Holders" for the "25-39" and "40-64" age groups of the "Bachelor's" dataset (df_2) for every State.
# NOTE: These age groups are considered to include both sexes ("Total" value of the "Sex" attribute).
# Filter the dataset to keep only rows where "Sex" is equal to "Total"
filtered_df = df_2[df_2["Sex"] == "Total"]

# Group the filtered DataFrame by "State"
grouped_df = filtered_df.groupby("State")

# Create an empty dictionary to store the results
state_sum_dict = {}

# Iterate over each distinct value of "State"
for state, group in grouped_df:
    # Filter the group based on the specified conditions and calculate the sum of "Bachelor's Degree Holders"
    age_group_condition = (group["Age Group"].isin(["25 to 39", "40 to 64"]))
    sum_value = group.loc[age_group_condition, "Bachelor's Degree Holders"].sum()

    # Save the sum to the state:sum dictionary
    state_sum_dict[state] = sum_value

df_1["#(Mid)Senior degree holders"] = df_1["State"].map(state_sum_dict)


# ====== Generate a new "Degree holders to establishments ratio" column that holds the
# #Degree Holders/#Business establishments ratio per State, taking into consideration both sexes.
# Group the df_2 DataFrame by "State" and filter rows where "Sex" is "Total"
filtered_df = df_2[df_2['Sex'] == 'Total'].groupby('State')

# Sum the values of "Age Group" per state
degree_holders_per_state = filtered_df['Bachelor\'s Degree Holders'].sum()

# Group the df_1 DataFrame by "State"
filtered_df = df_1.groupby("State")

# Sum the values of "Business size" per State
establishments_per_state = filtered_df["#Establishments"].sum()

# Calculate the ratio
ratio = degree_holders_per_state / establishments_per_state

# Create a new column in df_1 and map the #Degree Holders/#Business establishments ratios there,
# based on the "State" value of each entry.
df_1['Degree holders to establishments ratio'] = df_1['State'].map(ratio)


# For the Bachelor's dataset (df_2), drop any rows where "Sex" == "Total"
df_2 = df_2[(df_2["Sex"] != "Total")]


# Generate the analysis report
report_1 = sv.analyze(df_1)
report_2 = sv.analyze(df_2)

# Display the report in the browser
report_1.show_html('cbp_report.html')
report_2.show_html('bachelor_report.html')

print("> Saving preprocessed datasets to .csv files...")
df_1.to_csv('datasets/CBP_preprocessed.csv', index=False)
df_2.to_csv('datasets/Bachelor_preprocessed.csv', index=False)


> States included in CBP but not in Bachelor Majors dataset: 7
0            Maine
1      Puerto Rico
2           Alaska
3          Wyoming
4          Montana
5     South Dakota
6    West Virginia
dtype: object



Report cbp_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
Report bachelor_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
> Saving preprocessed datasets to .csv files...


### Adding new datasets

In [59]:
universities = pd.read_csv('datasets/National Universities Rankings.csv')
business = pd.read_csv('datasets/BDSTIMESERIES.BDSGEO-2023-05-31T192640.csv')
pd.set_option('display.max_columns', None)

Note: A firm is a business organization or entity consisting of one or more domestic establishments (locations) under common ownership or control. Dataset has firms and establishments, we look at establishments (also, dataset has only info about exited firms, not born)

Rate of establishments born during the last 12 months = The number of establishments born during last 12 months divided by the average number of estabs in year t (current year) and year t-1 (prior year)

Rate of establishments exited during the last 12 months = The number of establishments that exited during last 12 months divided by the average number of estabs in year t (current year) and year t-1 (prior year)

In [26]:
# Rename the columns of business to make them easier to work with
column_rename_mapping = {
    "Geographic Area Name (NAME)": "State",
    "Year (YEAR)": "Year",
    "Rate of establishments born during the last 12 months (ESTABS_ENTRY_RATE)": "Rate establishments born",
    "Rate of establishments exited during the last 12 months (ESTABS_EXIT_RATE)": "Rate establishments exited",
}
business.rename(columns=column_rename_mapping, inplace=True)

In [27]:
business = business[['State', 'Year', "Rate establishments born", "Rate establishments exited"]]

In [28]:
# Select the data for the last decade
business_recent = business[(business['Year'] >= 2009) & (business['Year'] <= 2019)]

In [32]:
# Calculate average rates for the last decade
business_agg = business_recent.groupby('State')[['Rate establishments born', 'Rate establishments exited']].mean()
business_agg.reset_index(inplace = True)

In [34]:
# Calculate the difference between born and exited rate (kind of "clean" born rate)
# If negative: more exited than born
business_agg['Rate born - exited'] = business_agg['Rate establishments born'] - business_agg['Rate establishments exited']

In [42]:
len(df_1['State'].unique().tolist()), len(business_agg['State'].unique().tolist())

(45, 51)

In [41]:
# Get the non-common values between the "State" columns of the 2 datasets (df_1 and business)
symmetric_difference = pd.Series(list(set(business_agg['State']).symmetric_difference(set(df_1['State']))))
print("\n> States included in Business Dynamics but not in CBP dataset: {}".format(len(symmetric_difference)))
print(symmetric_difference)


> States included in Business Dynamics but not in CBP dataset: 6
0            Maine
1           Alaska
2          Wyoming
3          Montana
4     South Dakota
5    West Virginia
dtype: object


In [43]:
# We will to drop the rows in business_agg that contain any of the non-common (State) values.
business_agg = business_agg[~business_agg['State'].isin(symmetric_difference.tolist())]

In [46]:
# Add dataset with states abbreviatios to work with universities ranking
states = pd.read_csv('datasets/state_names.csv')

In [60]:
universities['State Abbr'] = universities['Location'].str[-2:]

In [63]:
merged = pd.merge(universities, states, left_on='State Abbr', right_on = 'Alpha code')

In [66]:
# Add full state name
universities['State'] = merged['State']

In [68]:
# Select necessary columns
universities = universities[['Name', 'Rank', 'State']]

In [69]:
symmetric_difference = pd.Series(list(set(universities['State']).symmetric_difference(set(df_1['State']))))
print("\n> States included in Universities but not in CBP dataset: {}".format(len(symmetric_difference)))
print(symmetric_difference)


> States included in Universities but not in CBP dataset: 6
0            Maine
1           Alaska
2          Wyoming
3          Montana
4     South Dakota
5    West Virginia
dtype: object


In [71]:
# We will to drop the rows in universities that contain any of the non-common (State) values.
universities = universities[~universities['State'].isin(symmetric_difference.tolist())]

In [91]:
# Average, max and min rating of universities in a state: the LOWER the better (because rating starts at 1 = best university)
universities_agg = universities.groupby('State')[['Rank']].agg({'mean', 'max', 'min'})
universities_agg.reset_index(inplace = True)
universities_agg.columns = universities_agg.columns.droplevel()

In [94]:
universities_agg.rename(columns={"mean": "Average rank", 'max': 'Max rank', 'min': 'Min rank', '': 'State'}, inplace=True)

In [103]:
sorted_df = universities_agg.sort_values('Min rank')
# Top 10 states based on ranking
best_states = sorted_df.head(10)['State'].tolist()

In [104]:
universities_agg['State with top universities'] = universities_agg.apply(lambda x: 'Yes' if x['State'] in best_states else 'No', axis=1)

In [108]:
# Join business dynamics and universities datasets
final_extra = pd.merge(business_agg, universities_agg, on = 'State')

In [109]:
final_extra

,State,Rate establishments born,Rate establishments exited,Rate born - exited,Min rank,Max rank,Average rank,State with top universities
0,Alabama,8.094364,8.360364,-0.266000,197,202,199.500000,No
1,Arizona,10.948182,10.689182,0.259000,202,210,206.000000,No
2,Arkansas,8.583182,8.523091,0.060091,210,210,210.000000,No
3,California,10.854000,10.185727,0.668273,56,79,66.909091,Yes
4,Colorado,11.497455,10.652545,0.844909,188,188,188.000000,No
5,Connecticut,8.314545,8.707455,-0.392909,34,36,34.666667,Yes
6,Delaware,9.752455,9.424727,0.327727,188,188,188.000000,No
7,District of Columbia,9.878000,8.755091,1.122909,135,135,135.000000,No
8,Florida,12.417545,11.378727,1.038818,164,171,169.428571,No
9,Georgia,10.499727,10.142000,0.357727,133,135,134.500000,No


In [110]:
final_extra.to_csv('datasets/extra_datasets_preprocessed.csv', index=False)